In [1]:
# !git clone -b master https://github.com/StefanoNasini/Country-by-country-2022-2023-international-relationships
# !cd Country-by-country-2022-2023-international-relationships

In [2]:
import os
root = os.path.dirname(os.getcwd())
import pandas as pd
pd.set_option('display.max_rows', None)

In [12]:
#retrieving raw data

lgbt_full = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "lgbt_legal_status.csv"), index_col=0)
env_full = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "environmental.csv"), index_col = 0)
mig_full = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "migration_score.csv"), index_col=0)
# for palestine-israel alignment, it will be directly implemented in place in following sections

covotes = pd.read_csv(os.path.join(root, "constructed_network", "final_network", "friends80_bi.csv"), index_col=0)

In [13]:
# importing useful functions, mainly to parse full country names into trigrams

os.chdir(os.path.join(root, 'src', 'utils'))
from aka import tri2aka, tri2name, name2tri, guess_country
os.chdir(root)

In [14]:
# for migration, the index are full names
mig_full.index[:5]

Index(['new zealand', 'sweden', 'brazil', 'ireland', 'portugal'], dtype='object')

# trigramize the index 

In [15]:

# inspect how many countries need manual reassigning

to_rename = {}

for c in list(lgbt_full.index) + list(env_full.index) + list(mig_full.index):
    cc = c.replace('-', ' ')
    try:
        tri = name2tri[cc]
    except KeyError:
        try:
            tri = guess_country(cc)
        except Exception:
            tri = None
    to_rename[c] = tri
    
# print(to_rename)

In [16]:
remains = dict((k, v) for (k, v) in to_rename.items() if v==None)
print(remains)

{'cote-divoire': None, 'republic-of-the-congo': None, 'Timor-Leste': None, 'USA': None, 'korea': None}


In [17]:
# manually rename the country

to_rename['cote-divoire'] = "CIV"
to_rename['republic-of-the-congo'] = "COG"
to_rename['Timor-Leste'] = "TLS"
to_rename['Republic of the Congo'] = "COG"
to_rename['CÃ´te dâ\x80\x99Ivoire'] = "CIV"

to_rename['korea'] = "KOR"
to_rename['USA'] = "USA"

to_rename['R‚union'] = guess_country("reunion")
to_rename['China, Hong Kong SAR'] = "HKG"
to_rename['China, Macao SAR'] = "MAC"
to_rename['Cura‡ao'] = "CUW"
to_rename['United States Virgin Islands'] = "VIR"
try:
    # this country is not in UN so we drop it
    mig_full.drop("Channel Islands", axis='index', inplace=True)
except KeyError:
    pass

to_rename["guinea-bissau"] = "GNB"
to_rename["Guinea-Bissau"] = "GNB"

In [18]:
# create dataframes and use the trigram as the index

for c in lgbt_full.index:
    lgbt_full.loc[c, "tri"] = to_rename[c]
lgbt = lgbt_full.set_index("tri")

for c in env_full.index:
    env_full.loc[c, "tri"] = to_rename[c]
env = env_full.set_index("tri")

for c in mig_full.index:
    mig_full.loc[c, "tri"] = to_rename[c]
mig = mig_full.set_index("tri")

# UN country selection

In [19]:
# We only select the countries inside UN

# the UN countries trigrams
UN_tri = covotes.columns

# selection 
lgbt = lgbt.loc[lgbt.index.isin(UN_tri), :]
env = env.loc[env.index.isin(UN_tri), :]
mig = mig.loc[mig.index.isin(UN_tri), :]

# Missing data inspection

In [20]:
# The missing data for those countries in adjust growth rate for carbon dioxide
env_full.index[env_full["CDA"].isna()]

Index(['Zambia', 'Tajikistan', 'Bhutan', 'Laos', 'Cote d'Ivoire', 'Mongolia',
       'Ethiopia', 'Mozambique', 'Sri Lanka', 'Rwanda', 'Nepal', 'Cambodia',
       'Madagascar', 'Viet Nam', 'Fiji', 'Myanmar'],
      dtype='object')

In [21]:
# replace missing value with the mean
env["CDA"].fillna(env["CDA"].mean(), inplace=True)

In [22]:
# no missing for lgbt
lgbt.isna().sum()

Homosexuality                     0
Gay Marriage                      0
Censorship                        1
Changing Gender                   0
Gender-Affirming Care             0
Non-Binary Gender Recognition     0
Discrimination                    0
Employment Discrimination         0
Housing Discrimination            0
Adoption                          1
Intersex Infant Surgery           0
Military                         14
Donating Blood                    2
Conversion Therapy                0
Age of Consent                   48
dtype: int64

In [23]:
# missing data for migration flow are replaced with median
for tri in UN_tri:
    for item in mig.columns:
        if tri not in mig.index or mig.isna().loc[tri, item]:
            mig.loc[tri, item] = mig[item].median()

In [24]:
mig.head()

,access to nationality,anti discrimination,education,family reunion,health,labour market mobility,permanent residence,political participation
tri,,,,,,,,
NZL,92.0,88.0,76.0,74.0,83.0,59.0,63.0,85.0
SWE,83.0,100.0,93.0,71.0,83.0,91.0,90.0,80.0
BRA,91.0,85.0,14.0,94.0,31.0,67.0,96.0,35.0
IRL,79.0,94.0,45.0,48.0,85.0,22.0,50.0,85.0
PRT,86.0,100.0,69.0,87.0,65.0,94.0,71.0,80.0


# Prerequisite generation

## LGBT 


1. Death penalty for prerequisite 0

2. Legal adoption and Illegal discrimination 

3. Having a long history of legistration of same-sex marriage, at least earlier than 2010. Which involves
c.f. https://en.wikipedia.org/wiki/Timeline_of_same-sex_marriage

By filtering the 2 and 3 criteria, we will get a smaller set of prerequisites-to-1 countries. 

For real choice, we decide upon whether Homosexuality is legal or not.

In [25]:
prerequisite = pd.DataFrame(index= UN_tri, dtype=int)
prerequisite.loc[:, ["LGBT", "environmental", "migration", "IPalignment"]] = -1

real_choice = pd.DataFrame(index= UN_tri, dtype=int)
real_choice.loc[:, ["LGBT", "environmental", "migration", "IPalignment"]] = -1

significant_choice = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice.loc[:, ["LGBT", "environmental", "migration", "IPalignment"]] = -1

significant_choice70 = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice70.loc[:, ["LGBT", "environmental", "migration", "IPalignment"]] = -1

significant_choice80 = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice80.loc[:, ["LGBT", "environmental", "migration", "IPalignment"]] = -1

significant_choice85 = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice85.loc[:, ["LGBT", "environmental", "migration", "IPalignment"]] = -1

In [26]:
# we consider the real choice of countries to be 1 if the homosexuality is legal
real_choice["LGBT"] = 0
for tri in lgbt.index:
    if lgbt.loc[tri, "Homosexuality"] == '✔ Legal':
        real_choice.loc[tri, "LGBT"] = 1

In [27]:
# inspect countries where homosexuality is illegal
illegals = ["✖ Illegal (up to life in prison as punishment)", "✖ Illegal (death penalty as punishment)", "✖ Illegal (imprisonment as punishment)"]
lgbt_full[lgbt_full["Homosexuality"].isin(illegals)]

,Homosexuality,Gay Marriage,Censorship,Changing Gender,Gender-Affirming Care,Non-Binary Gender Recognition,Discrimination,Employment Discrimination,Housing Discrimination,Adoption,Intersex Infant Surgery,Military,Donating Blood,Conversion Therapy,Age of Consent,tri
iraq,✖ Illegal (imprisonment as punishment),✖ Unrecognized,✖ Imprisonment as punishment,✖ Illegal,✖ Banned,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Illegal,Unknown,✖ Illegal,✖ Banned (1-year deferral),✖ Not banned,NaN,IRQ
kuwait,✖ Illegal (imprisonment as punishment),✖ Banned,✖ State-enforced,✖ Illegal,✔ Legal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,NaN,Unknown,"✖ Don't Ask, Don't Tell",✔ Legal,✖ Not banned,NaN,KWT
malaysia,✖ Illegal (imprisonment as punishment),✖ Banned,✖ State-enforced,✖ Illegal,✖ Restricted,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Single only,Unknown,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,MYS
maldives,✖ Illegal (imprisonment as punishment),✖ Banned,✔ No censorship,✖ Illegal,✔ Legal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Single only,Unknown,"✖ Don't Ask, Don't Tell",✔ Legal,✖ Not banned,NaN,MDV
myanmar,✖ Illegal (imprisonment as punishment),✖ Unrecognized,✔ No censorship,✖ Illegal,✖ Restricted,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Single only,Unknown,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,MYR
oman,✖ Illegal (imprisonment as punishment),✖ Banned,✖ State-enforced,✖ Illegal,✔ Legal,✖ Not legally recognized,✖ No protections,Ambiguous,✖ No protections,✖ Illegal,Unknown,Ambiguous,✖ Banned (indefinite deferral),✖ Not banned,NaN,OMN
pakistan,✖ Illegal (imprisonment as punishment),✖ Banned,✖ Imprisonment as punishment,"✔ Legal, no restrictions","✖ Legal, but restricted for minors",✔ Recognized,✖ Illegal in some contexts,✖ Gender identity only,✖ Gender identity only,✖ Illegal,Unknown,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,PAK
qatar,✖ Illegal (imprisonment as punishment),✖ Banned,✖ State-enforced,✖ Illegal,✔ Legal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Illegal,Unknown,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,QAT
sri-lanka,✖ Illegal (imprisonment as punishment),✖ Banned,✔ No censorship,"✖ Legal, but requires medical diagnosis",✔ Legal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Illegal,Unknown,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,LKA
syria,✖ Illegal (imprisonment as punishment),✖ Banned,✖ State-enforced,"✖ Legal, but requires surgery",✔ Legal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Single only,Unknown,"✖ Don't Ask, Don't Tell",✖ Banned (indefinite deferral),✖ Not banned,NaN,SYR


In [28]:
# prerequisite is set to 0 for those countries where homosexuality is illegal

for tri in lgbt[lgbt["Homosexuality"].isin(illegals)].index:
    prerequisite.loc[tri, "LGBT"] = 0

In [29]:
# we set prerequisite to be 1 for those countries where gay marriage is legal and the adoption is legal
for tri in lgbt.index:
    if (lgbt.loc[tri, "Gay Marriage"] == "✔ Legal") and (lgbt.loc[tri, "Adoption"] == "✔ Legal"):
        prerequisite.loc[tri, "LGBT"] = 1
        print(tri2name[tri])

austria
belgium
denmark
estonia
finland
france
germany
greece
iceland
ireland
luxembourg
malta
netherlands
norway
portugal
slovenia
spain
sweden
switzerland
united kingdom
canada
costa rica
cuba
united states
south africa
australia
new zealand
argentina
brazil
chile
colombia
uruguay


In [30]:
# also for those 25 countries that are concretely open to homosexuality in terms of anti-discrimination and adoption. Anchor to 1. 

lgbt3 = map(guess_country, "Netherlands, spain, canada, belgium, south africa, norway, sweden, iceland".split(', '))
for tri in lgbt3:
    prerequisite.loc[tri, "LGBT"] = 1

    
# len(one), len(zero)

## Environmental
1. The adjusted growth rate of carbon dioxide emission (CDA). We select top 15 and bottom 15 as anchor nodes.

In [31]:
pd.set_option('display.max_rows', None)
env["CDA"].sort_values()

tri
IRQ      1.900000
BFA      2.200000
TON      2.300000
MRT      2.400000
PHL      2.800000
BDI      4.100000
LBR      4.200000
HTI      4.400000
MLI      5.400000
NER      5.400000
MDV      7.800000
TLS      8.100000
SDN      8.500000
BEN      8.800000
PAK      8.800000
COM      9.000000
BGD      9.400000
TZA     10.600000
BWA     10.700000
GHA     11.600000
GTM     12.200000
SLE     12.700000
WSM     13.100000
PNG     13.100000
UGA     15.400000
PRY     17.200000
TGO     17.400000
IND     17.600000
KIR     17.900000
GMB     17.900000
GNB     18.100000
SEN     18.200000
TCD     18.300000
NAM     18.700000
KEN     18.700000
DZA     18.800000
KGZ     20.500000
VUT     21.700000
BOL     22.700000
OMN     23.400000
ERI     23.400000
GIN     23.500000
GUY     25.100000
NIC     26.500000
TUR     26.500000
GEO     26.700000
MWI     27.000000
BLZ     27.300000
IDN     27.400000
PER     27.600000
HND     28.100000
IRN     28.200000
ARM     29.200000
ZWE     29.300000
TKM     29.500000
CMR   

In [32]:
import numpy as np

In [33]:
# we distinguish prerequiste (top or bottom 10), 
# significant choice (top or bottom 40%), significant choice 70 (top or bottom 30%), significant choice 80 (top or bottom 20%)
sig40, sig60 = np.percentile(env["CDA"].sort_values(), 40), np.percentile(env["CDA"].sort_values(), 60)
sig30, sig70 = np.percentile(env["CDA"].sort_values(), 30), np.percentile(env["CDA"].sort_values(), 70)
sig20, sig80 = np.percentile(env["CDA"].sort_values(), 20), np.percentile(env["CDA"].sort_values(), 80)
sig15, sig85 = np.percentile(env["CDA"].sort_values(), 15), np.percentile(env["CDA"].sort_values(), 85)


for i, tri in enumerate((env["CDA"].sort_values()).index): # increasing
    if i<10:
        prerequisite.loc[tri, "environmental"] = 0
    if env["CDA"].sort_values()[i] <= sig40:
        significant_choice.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values()[i] <= sig30:
        significant_choice70.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values()[i] <= sig20:
        significant_choice80.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values()[i] <= sig15:
        significant_choice80.loc[tri, "environmental"] = 1
        
for i, tri in enumerate(reversed((env["CDA"].sort_values()).index)):
    if i<10:
        prerequisite.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig60:
        significant_choice.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig70:
        significant_choice70.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig80:
        significant_choice80.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig85:
        significant_choice80.loc[tri, "environmental"] = 1
        

        
for i, tri in enumerate((env["CDA"].sort_values()).index):
    if 2*i<len(env.index):
        real_choice.loc[tri, "environmental"] = 0
    else:
        real_choice.loc[tri, "environmental"] = 1


In [34]:
prerequisite.loc[:, "environmental"] == 0

AFG    False
AGO    False
ALB    False
ARE    False
ARG    False
ARM    False
ATG    False
AUS    False
AUT    False
AZE    False
BDI     True
BEL    False
BEN    False
BFA     True
BGD    False
BGR    False
BHR    False
BHS    False
BIH    False
BLR    False
BLZ    False
BOL    False
BRA    False
BRB    False
BRN    False
BTN    False
BWA    False
CAF    False
CAN    False
CHE    False
CHL    False
CHN    False
CIV    False
CMR    False
COD    False
COG    False
COL    False
COM    False
CPV    False
CRI    False
CUB    False
CYP    False
CZE    False
DEU    False
DJI    False
DMA    False
DNK    False
DOM    False
DZA    False
ECU    False
EGY    False
ERI    False
ESP    False
EST    False
ETH    False
FIN    False
FJI    False
FRA    False
GAB    False
GBR    False
GEO    False
GHA    False
GIN    False
GMB    False
GNB    False
GNQ    False
GRC    False
GRD    False
GTM    False
GUY    False
HND    False
HRV    False
HTI     True
HUN    False
IDN    False
IND    False
IRL    False

## Equality
Look at Legal index. choose top 10 and bottom 10. 

In [35]:
# lb, ub = eq["Legal Index"].sort_values()[[9, -9]]
# siglb, sigub = np.percentile(eq["Legal Index"].sort_values(), 40), np.percentile(eq["Legal Index"].sort_values(), 60)
# siglb30, sigub70 = np.percentile(eq["Legal Index"].sort_values(), 30), np.percentile(eq["Legal Index"].sort_values(), 70)
# siglb20, sigub80 = np.percentile(eq["Legal Index"].sort_values(), 20), np.percentile(eq["Legal Index"].sort_values(), 80)
# siglb15, sigub85 = np.percentile(eq["Legal Index"].sort_values(), 15), np.percentile(eq["Legal Index"].sort_values(), 85)

In [36]:
# for i, tri in enumerate(eq.index):
#     if eq.loc[tri, "Legal Index"] <= lb:
#         prerequisite.loc[tri, "equality"] = 0
#     if eq["Legal Index"].sort_values()[i] <= siglb:
#         significant_choice.loc[tri, "equality"] = 1
#     if eq["Legal Index"].sort_values()[i] <= siglb30:
#         significant_choice70.loc[tri, "equality"] = 1
#     if eq["Legal Index"].sort_values()[i] <= siglb20:
#         significant_choice80.loc[tri, "equality"] = 1
#     if eq["Legal Index"].sort_values()[i] <= siglb15:
#         significant_choice85.loc[tri, "equality"] = 1

#     if eq.loc[tri, "Legal Index"] >= ub:
#         prerequisite.loc[tri, "equality"] = 1
#     if eq["Legal Index"].sort_values()[i] >= sigub:
#         significant_choice.loc[tri, "equality"] = 1
#     if eq["Legal Index"].sort_values()[i] >= sigub70:
#         significant_choice70.loc[tri, "equality"] = 1
#     if eq["Legal Index"].sort_values()[i] >= sigub80:
#         significant_choice80.loc[tri, "equality"] = 1
#     if eq["Legal Index"].sort_values()[i] >= sigub85:
#         significant_choice85.loc[tri, "equality"] = 1


In [37]:
(significant_choice == 1).sum(axis=0)

LGBT               0
environmental    141
migration          0
IPalignment        0
dtype: int64

In [38]:
# for i, tri in enumerate(eq.index):
#     if 2*i < len(eq.index):
#         real_choice.loc[tri, "equality"] = 0
#     else:
#         real_choice.loc[tri, "equality"] = 1

## Migration

For prerequisite 1 : Appear in top for more than four times and appear in bottom less than one time

For prerequisite 0 : Appear in top for less than one time and appear in bottom more than four times


In [39]:
count_pos = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos60 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg60 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos70 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg70 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos80 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg80 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos85 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg85 = dict(zip(UN_tri,[0]*len(UN_tri)) )

for item in mig.columns:
    for tri in UN_tri:
        if mig.loc[tri, item] >= mig[item].sort_values(ascending=False)[14]:
            count_pos[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 60):
            count_pos60[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 70):
            count_pos70[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 80):
            count_pos80[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 85):
            count_pos85[tri] += 1
            
        if mig.loc[tri, item] <= mig[item].sort_values(ascending=True)[14]:
            count_neg[tri] += 1
#         if mig.loc[tri, item] <= np.percentile(mig[item].sort_values(ascending=False), 40):
            count_neg60[tri] += 1
#         if mig.loc[tri, item] <= np.percentile(mig[item].sort_values(ascending=False), 30):
            count_neg70[tri] += 1
#         if mig.loc[tri, item] <= np.percentile(mig[item].sort_values(ascending=False), 20):
            count_neg80[tri] += 1
            count_neg85[tri] += 1

In [40]:
for tri in UN_tri:
    if mig.mean(axis=1).sort_values()[tri] >= mig.mean(axis=1).sort_values().mean():
        real_choice.loc[tri, "migration"] = 1
    else:
        real_choice.loc[tri, "migration"] = 0


In [41]:
# for tri in set(count_pos.keys()).union(set(count_neg.keys())):
#     if (tri in count_pos.keys() and count_pos[tri] >= 4) and (tri not in count_neg.keys() or count_neg[tri]<=1):
#         print(f"To prerequisite 1 is country {tri2name[tri]}")
#         prerequisite.loc[tri, 'migration'] = 1

# for tri in set(count_pos.keys()).union(set(count_neg.keys())):
#     if (tri in count_neg.keys() and count_neg[tri] >= 4) and (tri not in count_pos.keys() or count_pos[tri]<=1):
#         print(f"To prerequisite 0 is country {tri2name[tri]}")
#         prerequisite.loc[tri, 'migration'] = 0

In [42]:
guess_country("hungary")
tri = "HUN"
# (tri in count_neg.keys() and count_neg[tri] >= 4) and (tri not in count_pos.keys() or count_pos[tri]<=1)
count_pos["HUN"], count_neg["HUN"]

(2, 3)

In [43]:
# to accomodate Hungary, the criteria needs to be changed to >=3 and <=2

for tri in set(count_pos.keys()).union(set(count_neg.keys())):
    if (tri in count_pos.keys() and count_pos[tri] >= 3) and (tri not in count_neg.keys() or count_neg[tri]<=2):
        print(f"To prerequisite 1 is country {tri2name[tri]}")
        prerequisite.loc[tri, 'migration'] = 1
        significant_choice.loc[tri, 'migration'] = 1
        significant_choice70.loc[tri, 'migration'] = 1
        significant_choice80.loc[tri, 'migration'] = 1
        significant_choice85.loc[tri, 'migration'] = 1


    else:
        if (tri in count_pos60.keys() and count_pos60[tri] >= 3) and (tri not in count_neg60.keys() or count_neg60[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice.loc[tri, 'migration'] = 1
        if (tri in count_pos70.keys() and count_pos70[tri] >= 3) and (tri not in count_neg70.keys() or count_neg70[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice70.loc[tri, 'migration'] = 1
        if (tri in count_pos80.keys() and count_pos80[tri] >= 3) and (tri not in count_neg80.keys() or count_neg80[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice80.loc[tri, 'migration'] = 1
        if (tri in count_pos85.keys() and count_pos85[tri] >= 3) and (tri not in count_neg85.keys() or count_neg85[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice85.loc[tri, 'migration'] = 1
        
    if (tri in count_neg.keys() and count_neg[tri] >= 3) and (tri not in count_pos.keys() or count_pos[tri]<=2):
        print(f"To prerequisite 0 is country {tri2name[tri]}")
        prerequisite.loc[tri, 'migration'] = 0
        significant_choice.loc[tri, 'migration'] = 0
        significant_choice70.loc[tri, 'migration'] = 1
        significant_choice80.loc[tri, 'migration'] = 1
        significant_choice85.loc[tri, 'migration'] = 1

    else:
        if (tri in count_neg60.keys() and count_neg60[tri] >= 3) and (tri not in count_pos60.keys() or count_pos60[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice.loc[tri, 'migration'] = 0
        if (tri in count_neg70.keys() and count_neg70[tri] >= 3) and (tri not in count_pos70.keys() or count_pos70[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice70.loc[tri, 'migration'] = 0
        if (tri in count_neg80.keys() and count_neg80[tri] >= 3) and (tri not in count_pos80.keys() or count_pos80[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice80.loc[tri, 'migration'] = 0
        if (tri in count_neg85.keys() and count_neg85[tri] >= 3) and (tri not in count_pos85.keys() or count_pos85[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice85.loc[tri, 'migration'] = 0

To significant 1 is country democratic people's republic of korea
To significant 1 is country democratic people's republic of korea
To significant 1 is country democratic people's republic of korea
To significant 1 is country democratic people's republic of korea
To significant 1 is country georgia
To significant 1 is country georgia
To significant 1 is country georgia
To significant 1 is country georgia
To significant 1 is country kuwait
To significant 1 is country kuwait
To significant 1 is country kuwait
To significant 1 is country kuwait
To significant 1 is country dominican republic
To significant 1 is country dominican republic
To significant 1 is country dominican republic
To significant 1 is country dominican republic
To significant 1 is country eritrea
To significant 1 is country eritrea
To significant 1 is country eritrea
To significant 1 is country eritrea
To significant 1 is country united kingdom
To significant 1 is country united kingdom
To significant 1 is country united

## Israel-Palestine alignment
To prerequisite 0 are countries who have not recognize Palestine as a state

Israel, united states, Canada, Australia, Austria, Bulgaria, Croatia, Cyprus, 
Czech Republic, Estonia, Greece, Hungary, Latvia, Lithuania, Poland, Romania, Slovakia, Germany, 
Italy, Japan, South Korea, UK

To Prerequisite 1 are countries who don’t recognize Israel as a state:

Afghanistan, Algeria, Bahrain, Bangladesh, Brunei, Comoros, Djibouti, Iran, Iraq, Kuwait, Lebanon, 
Libya, Malaysia, Mauritania, Morocco, North Korea, Oman, Pakistan, Qatar, Saudi Arabia, Somalia, 
Sudan, Syria, Tunisia, United Arab Emirates, Yemen.

In [44]:
IPzero = """Israel, united states, Canada, Australia, Austria, Bulgaria, Croatia, Cyprus, 
Czech Republic, Estonia, Greece, Hungary, Latvia, Lithuania, Poland, Romania, Slovakia, Germany, 
Italy, Japan, South Korea, UK""".split(", ")

IPzero = list(map(guess_country, IPzero))
for tri in IPzero:
    prerequisite.loc[tri, "IPalignment"] = 0

In [45]:
IPone = """Afghanistan, Algeria, Bahrain, Bangladesh, Brunei, Comoros, Djibouti, Iran, Iraq, Kuwait, Lebanon, 
Libya, Malaysia, Mauritania, Morocco, North Korea, Oman, Pakistan, Qatar, Saudi Arabia, Somalia, 
Sudan, Syria, Tunisia, United Arab Emirates, Yemen""".split(", ")

IPone = list(map(guess_country, IPone))
for tri in IPone:
    prerequisite.loc[tri, "IPalignment"] = 1

In [46]:
# for i, tri in enumerate((mig["2015"].sort_values()).index):
#     if i<10:
#         prerequisite.loc[tri, "migration"] = 0
#         print(tri2name[tri])
#     else:
#         break
        
# print()
        
# for i, tri in enumerate(reversed((mig["2015"].sort_values()).index)):
#     if i<10:
#         prerequisite.loc[tri, "migration"] = 1
#         print(tri2name[tri])
#     else:
#         break
        
        
# for i, tri in enumerate((mig["2015"].sort_values()).index):
#     if 2*i<len(mig.index):
#         real_choice.loc[tri, "migration"] = 0
#     else:
#         real_choice.loc[tri, "migration"] = 1

In [47]:
significant_choice

,LGBT,environmental,migration,IPalignment
AFG,-1.0,1.0,1.0,-1.0
AGO,-1.0,-1.0,1.0,-1.0
ALB,-1.0,-1.0,0.0,-1.0
ARE,-1.0,-1.0,0.0,-1.0
ARG,-1.0,1.0,1.0,-1.0
ARM,-1.0,1.0,1.0,-1.0
ATG,-1.0,-1.0,1.0,-1.0
AUS,-1.0,1.0,1.0,-1.0
AUT,-1.0,1.0,0.0,-1.0
AZE,-1.0,1.0,1.0,-1.0


In [48]:
with open(os.path.join(root, 'constructed_network', "predefined.txt"), 'w') as f:
    for i in range(prerequisite.shape[0]):
        for j in range(prerequisite.shape[1]):
            try:
                f.write(str(int(prerequisite.iloc[i, j])))
            except ValueError:
                pass
            f.write(' ')
        f.write("\n")

In [49]:
# import random

# random.seed(0)
# for i in range(prerequisite.shape[0]):
#     for j in range(prerequisite.shape[1]):
#         if random.random() < 0.9:
#             prerequisite.iloc[i, j] = random.randint(0, 1)
#         else:
#             prerequisite.iloc[i, j] = -1

In [50]:
(significant_choice80 == 1).sum()

LGBT               0
environmental     72
migration        181
IPalignment        0
dtype: int64

In [51]:
real_choice.isna().sum()

LGBT             0
environmental    0
migration        0
IPalignment      0
dtype: int64

In [52]:
real_choice.to_csv(os.path.join(root, "constructed_network", "v_real_choice_4items.csv"),float_format='%.0f')
prerequisite.to_csv(os.path.join(root, "constructed_network", "prerequisite_4items.csv"),float_format='%.0f')
significant_choice.to_csv(os.path.join(root, "constructed_network", "significant_choice_4items.csv"),float_format='%.0f')
significant_choice70.to_csv(os.path.join(root, "constructed_network", "significant_choice70_4items.csv"),float_format='%.0f')
significant_choice80.to_csv(os.path.join(root, "constructed_network", "significant_choice80_4items.csv"),float_format='%.0f')
significant_choice85.to_csv(os.path.join(root, "constructed_network", "significant_choice85_4items.csv"),float_format='%.0f')



# real_choice.drop(columns=['equality']).to_csv(os.path.join(root, "constructed_network", "v_real_choice_4items.csv"))
# prerequisite.drop(columns=['equality']).to_csv(os.path.join(root, "constructed_network", "prerequisite_4items.csv"))
# significant_choice.drop(columns=['equality']).to_csv(os.path.join(root, "constructed_network", "significant_choice_4items.csv"))
# significant_choice70.drop(columns=['equality']).to_csv(os.path.join(root, "constructed_network", "significant_choice70_4items.csv"))
# significant_choice80.drop(columns=['equality']).to_csv(os.path.join(root, "constructed_network", "significant_choice80_4items.csv"))
# significant_choice85.drop(columns=['equality']).to_csv(os.path.join(root, "constructed_network", "significant_choice85_4items.csv"))

# prerequisite.drop(columns=['equality']).to_csv(os.path.join(root, "constructed_network", "v_predefined_4items.txt"), sep=' ', header=False, index=False)

In [53]:
(prerequisite==-1).sum()

LGBT             108
environmental    164
migration        146
IPalignment      136
dtype: int64

In [54]:
for i, c in enumerate(prerequisite['LGBT']):
    print(f"fixed to {c} is country {i}") if int(c) != -1 else None

fixed to 0.0 is country 0
fixed to 0.0 is country 3
fixed to 1.0 is country 4
fixed to 1.0 is country 7
fixed to 1.0 is country 8
fixed to 0.0 is country 10
fixed to 1.0 is country 11
fixed to 0.0 is country 14
fixed to 1.0 is country 22
fixed to 0.0 is country 24
fixed to 1.0 is country 28
fixed to 1.0 is country 29
fixed to 1.0 is country 30
fixed to 0.0 is country 33
fixed to 1.0 is country 36
fixed to 0.0 is country 37
fixed to 1.0 is country 39
fixed to 1.0 is country 40
fixed to 1.0 is country 43
fixed to 1.0 is country 46
fixed to 0.0 is country 48
fixed to 0.0 is country 51
fixed to 1.0 is country 52
fixed to 1.0 is country 53
fixed to 0.0 is country 54
fixed to 1.0 is country 55
fixed to 1.0 is country 57
fixed to 1.0 is country 59
fixed to 0.0 is country 61
fixed to 0.0 is country 62
fixed to 0.0 is country 63
fixed to 1.0 is country 66
fixed to 1.0 is country 76
fixed to 0.0 is country 77
fixed to 0.0 is country 78
fixed to 1.0 is country 79
fixed to 0.0 is country 86
fixed 

In [55]:
real_choice['LGBT'].mean()

0.6521739130434783